In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner


!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting map library

print('Folium installed')
print('All Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium installed
All Libraries imported.


In [2]:
CLIENT_ID='NSWC0SRL1NVQY551AP3I0RAQPIFKF3PXC3XBYGT1DEGC0CJM'
CLIENT_SECRET='1EDCF3ZSYUHAO2A03AOK0QEIIWM1UGSL3XBSYKTV4LYHAWV2'
latitude=40.7590
longitude=-73.9845
VERSION=20200401
search_query = 'restaurants'
radius = 10000 # 10 KM
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius)
result_restaurants = requests.get(url).json()
result_restaurants

{'meta': {'code': 200, 'requestId': '5e9710b51835dd001bf1f96e'},
 'response': {'venues': [{'id': '5b9f8c9833e118002c53aa87',
    'name': 'The Shops & Restaurants at Hudson Yards',
    'location': {'address': '20 Hudson Yards',
     'lat': 40.75368434864726,
     'lng': -74.00195360183716,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.75368434864726,
       'lng': -74.00195360183716}],
     'distance': 1586,
     'postalCode': '10001',
     'cc': 'US',
     'neighborhood': 'Hudson Yards',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['20 Hudson Yards',
      'New York, NY 10001',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1fd941735',
      'name': 'Shopping Mall',
      'pluralName': 'Shopping Malls',
      'shortName': 'Mall',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/mall_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1586958714',
 

In [3]:
# assign relevant part of JSON to hotel
restaurants = result_restaurants['response']['venues']

# tranform hotels into a dataframe
df_restaurants = json_normalize(restaurants)
df_restaurants.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,...,location.country,location.formattedAddress,location.crossStreet,venuePage.id,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,5b9f8c9833e118002c53aa87,The Shops & Restaurants at Hudson Yards,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1586958714,False,20 Hudson Yards,40.753684,-74.001954,"[{'label': 'display', 'lat': 40.75368434864726...",1586,...,United States,"[20 Hudson Yards, New York, NY 10001, United S...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,528b65eb498e12f0376394bd,Barilla Restaurants,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1586958714,False,1290 Avenue of the Americas,40.761146,-73.979155,"[{'label': 'display', 'lat': 40.76114629747720...",510,...,United States,[1290 Avenue of the Americas (6th Ave @ 52nd (...,6th Ave @ 52nd (entrance on 52nd),NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5127ca86e4b0f46f576a637f,Dallas BBQ Restaurants,"[{'id': '4bf58dd8d48988d1cc941735', 'name': 'S...",v-1586958714,False,NaN,40.757000,-73.989126,"[{'label': 'display', 'lat': 40.757, 'lng': -7...",449,...,United States,"[New York, NY, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4f329e3719836c91c7e94f57,Wu Kong Restaurants,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",v-1586958714,False,30 W 46th St,40.756473,-73.980514,"[{'label': 'display', 'lat': 40.756473, 'lng':...",438,...,United States,"[30 W 46th St, New York, NY 10036, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4be5ba59cf200f47f3a0133c,Go Burger by BLT restaurants Group,[],v-1586958714,False,907,40.764954,-73.981950,"[{'label': 'display', 'lat': 40.76495415857458...",696,...,United States,"[907 (20 / 21 at), New York, NY, United States]",20 / 21 at,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# keep only columns that include hotel name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df_restaurants.columns if col.startswith('location.')] + ['id']
df_restaurants_filtered = df_restaurants.loc[:, filtered_columns]

# function that extracts the category of the hotels
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_restaurants_filtered['categories'] = df_restaurants_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_restaurants_filtered.columns = [column.split('.')[-1] for column in df_restaurants_filtered.columns]

pd.DataFrame(df_restaurants_filtered)
df_restaurants_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,crossStreet,id
0,The Shops & Restaurants at Hudson Yards,Shopping Mall,20 Hudson Yards,40.753684,-74.001954,"[{'label': 'display', 'lat': 40.75368434864726...",1586,10001,US,Hudson Yards,New York,NY,United States,"[20 Hudson Yards, New York, NY 10001, United S...",NaN,5b9f8c9833e118002c53aa87
1,Barilla Restaurants,Italian Restaurant,1290 Avenue of the Americas,40.761146,-73.979155,"[{'label': 'display', 'lat': 40.76114629747720...",510,10104,US,"Midtown Manhattan, New York, NY",New York,NY,United States,[1290 Avenue of the Americas (6th Ave @ 52nd (...,6th Ave @ 52nd (entrance on 52nd),528b65eb498e12f0376394bd
2,Dallas BBQ Restaurants,Steakhouse,NaN,40.757000,-73.989126,"[{'label': 'display', 'lat': 40.757, 'lng': -7...",449,NaN,US,NaN,New York,NY,United States,"[New York, NY, United States]",NaN,5127ca86e4b0f46f576a637f
3,Wu Kong Restaurants,Food,30 W 46th St,40.756473,-73.980514,"[{'label': 'display', 'lat': 40.756473, 'lng':...",438,10036,US,NaN,New York,NY,United States,"[30 W 46th St, New York, NY 10036, United States]",NaN,4f329e3719836c91c7e94f57
4,Go Burger by BLT restaurants Group,None,907,40.764954,-73.981950,"[{'label': 'display', 'lat': 40.76495415857458...",696,NaN,US,NaN,New York,NY,United States,"[907 (20 / 21 at), New York, NY, United States]",20 / 21 at,4be5ba59cf200f47f3a0133c
5,Saigon Restaurants Ny,Food,6 E 37th St,40.749848,-73.982589,"[{'label': 'display', 'lat': 40.749848, 'lng':...",1031,10016,US,NaN,New York,NY,United States,"[6 E 37th St, New York, NY 10016, United States]",NaN,4f3253cb19836c91c7ccd5e7
6,Fourth Wall Restaurants,Food,880 3rd Ave,40.757902,-73.969248,"[{'label': 'display', 'lat': 40.757902, 'lng':...",1291,10022,US,NaN,New York,NY,United States,"[880 3rd Ave, New York, NY 10022, United States]",NaN,4f32acc619836c91c7eec29f
7,Top 10 Restaurants New York,Restaurant,244 5th Ave Fl 2,40.744665,-73.987446,"[{'label': 'display', 'lat': 40.74466505618245...",1614,10001,US,NaN,New York,NY,United States,"[244 5th Ave Fl 2, New York, NY 10001, United ...",NaN,4eefb134d5fbe9f4b1f2054d
8,BOLD restaurants Office,None,NaN,40.744558,-73.987548,"[{'label': 'display', 'lat': 40.744558, 'lng':...",1628,10001,US,NaN,New York,NY,United States,"[New York, NY 10001, United States]",NaN,4c728019d944224b97c60c29
9,Restaurants Open 24,Falafel Restaurant,NaN,40.724090,-73.978742,"[{'label': 'display', 'lat': 40.7240899122677,...",3916,NaN,US,NaN,New York,NY,United States,"[New York, NY, United States]",NaN,519c3b8c498e55e13a4e3c1b


In [7]:
df_restaurants_filtered = df_restaurants_filtered[df_restaurants_filtered.categories.isin(['Food','Mexican Restaurant','Irish Pub', 'Sandwich Place', 'Italian Restaurant','Steakhouse'])]
df_restaurants_filtered.categories = ('Food')
df_restaurants_filtered.reset_index(drop=True, inplace=True)
df_restaurants_filtered

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,crossStreet,id
0,Barilla Restaurants,Food,1290 Avenue of the Americas,40.761146,-73.979155,"[{'label': 'display', 'lat': 40.76114629747720...",510,10104,US,"Midtown Manhattan, New York, NY",New York,NY,United States,[1290 Avenue of the Americas (6th Ave @ 52nd (...,6th Ave @ 52nd (entrance on 52nd),528b65eb498e12f0376394bd
1,Dallas BBQ Restaurants,Food,NaN,40.757000,-73.989126,"[{'label': 'display', 'lat': 40.757, 'lng': -7...",449,NaN,US,NaN,New York,NY,United States,"[New York, NY, United States]",NaN,5127ca86e4b0f46f576a637f
2,Wu Kong Restaurants,Food,30 W 46th St,40.756473,-73.980514,"[{'label': 'display', 'lat': 40.756473, 'lng':...",438,10036,US,NaN,New York,NY,United States,"[30 W 46th St, New York, NY 10036, United States]",NaN,4f329e3719836c91c7e94f57
3,Saigon Restaurants Ny,Food,6 E 37th St,40.749848,-73.982589,"[{'label': 'display', 'lat': 40.749848, 'lng':...",1031,10016,US,NaN,New York,NY,United States,"[6 E 37th St, New York, NY 10016, United States]",NaN,4f3253cb19836c91c7ccd5e7
4,Fourth Wall Restaurants,Food,880 3rd Ave,40.757902,-73.969248,"[{'label': 'display', 'lat': 40.757902, 'lng':...",1291,10022,US,NaN,New York,NY,United States,"[880 3rd Ave, New York, NY 10022, United States]",NaN,4f32acc619836c91c7eec29f
5,Lucy Restaurants,Food,35 E 18th St,40.737776,-73.990076,"[{'label': 'display', 'lat': 40.737776, 'lng':...",2408,10003,US,NaN,New York,NY,United States,"[35 E 18th St, New York, NY 10003, United States]",NaN,4f32b6f819836c91c7f28b59
6,Restaurant SX,Food,247 Smith St,40.754363,-73.967016,"[{'label': 'display', 'lat': 40.75436331349777...",1562,11231,US,NaN,Brooklyn,NY,United States,"[247 Smith St, Brooklyn, NY 11231, United States]",NaN,54e6baf8498edefbe56eeec3
7,Mezzaluna Restaurants,Food,1295 3rd Ave,40.771352,-73.959682,"[{'label': 'display', 'lat': 40.77135248875577...",2503,10021,US,NaN,New York,NY,United States,"[1295 3rd Ave (btwn E 74th & E 75th St), New Y...",btwn E 74th & E 75th St,4a78f65ef964a520b7e61fe3
8,Night and Day Restaurants,Food,401 37th St,40.774668,-74.027251,"[{'label': 'display', 'lat': 40.774668, 'lng':...",4004,07087,US,NaN,Union City,NJ,United States,"[401 37th St, Union City, NJ 07087, United Sta...",NaN,4f32323119836c91c7bf991f
9,Subway,Food,52 Vanderbilt Ave,40.754302,-73.977363,"[{'label': 'display', 'lat': 40.75430156359721...",797,10017,US,NaN,New York,NY,United States,"[52 Vanderbilt Ave, New York, NY 10017, United...",NaN,5a8c53c19fca567a0571cb65


In [8]:
df_restaurants_filtered.drop(['crossStreet', 'labeledLatLngs', 'postalCode', 'state', 'country', 'city'], axis=1, inplace=True)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [9]:
df_restaurants_filtered

,name,categories,address,lat,lng,distance,cc,neighborhood,formattedAddress,id
0,Barilla Restaurants,Food,1290 Avenue of the Americas,40.761146,-73.979155,510,US,"Midtown Manhattan, New York, NY",[1290 Avenue of the Americas (6th Ave @ 52nd (...,528b65eb498e12f0376394bd
1,Dallas BBQ Restaurants,Food,NaN,40.757000,-73.989126,449,US,NaN,"[New York, NY, United States]",5127ca86e4b0f46f576a637f
2,Wu Kong Restaurants,Food,30 W 46th St,40.756473,-73.980514,438,US,NaN,"[30 W 46th St, New York, NY 10036, United States]",4f329e3719836c91c7e94f57
3,Saigon Restaurants Ny,Food,6 E 37th St,40.749848,-73.982589,1031,US,NaN,"[6 E 37th St, New York, NY 10016, United States]",4f3253cb19836c91c7ccd5e7
4,Fourth Wall Restaurants,Food,880 3rd Ave,40.757902,-73.969248,1291,US,NaN,"[880 3rd Ave, New York, NY 10022, United States]",4f32acc619836c91c7eec29f
5,Lucy Restaurants,Food,35 E 18th St,40.737776,-73.990076,2408,US,NaN,"[35 E 18th St, New York, NY 10003, United States]",4f32b6f819836c91c7f28b59
6,Restaurant SX,Food,247 Smith St,40.754363,-73.967016,1562,US,NaN,"[247 Smith St, Brooklyn, NY 11231, United States]",54e6baf8498edefbe56eeec3
7,Mezzaluna Restaurants,Food,1295 3rd Ave,40.771352,-73.959682,2503,US,NaN,"[1295 3rd Ave (btwn E 74th & E 75th St), New Y...",4a78f65ef964a520b7e61fe3
8,Night and Day Restaurants,Food,401 37th St,40.774668,-74.027251,4004,US,NaN,"[401 37th St, Union City, NJ 07087, United Sta...",4f32323119836c91c7bf991f
9,Subway,Food,52 Vanderbilt Ave,40.754302,-73.977363,797,US,NaN,"[52 Vanderbilt Ave, New York, NY 10017, United...",5a8c53c19fca567a0571cb65


In [11]:
allrestaurants_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around the New York City

#add outline of theatre district
coordinates = [
    [40.764346,-73.983779],
    [40.759103,-73.983090],
    [40.755484,-73.988302],
    [40.759253,-73.989211],
    [40.764346,-73.983779]
]

my_PolyLine=folium.PolyLine(locations=coordinates,weight=5)
allrestaurants_map.add_child(my_PolyLine)

# add the Hotels as blue circle markers
for lat, lng, label in zip(df_restaurants_filtered.lat, df_restaurants_filtered.lng, df_restaurants_filtered.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(allrestaurants_map)

# display map
allrestaurants_map